In [1]:
from pycalphad import Database, variables as v
from pycalphad.variables import Species

dbf = Database('Al-Cu-Zr_Zhou.tdb')

In [2]:
import itertools
import string
import sympy
import numpy as np

def genericize(mapping, const_array):
    new_array = []
    for subl in const_array:
        new_array.append(tuple(mapping[x] for x in subl))
    return tuple(new_array)
def sigfigs(x, n):
    """Round x to n significant digits"""
    if x != 0:
        return np.around(x, -(np.floor(np.log10(np.abs(x)))).astype(np.int) + (n - 1))
    else:
        return x
def average_piecewise(pw):
    return sum(x.expr for x in pw.args)/len(pw.args)

def categorize_phase(phase_name, constituents, site_ratios):
    if phase_name == 'LIQUID' and len(constituents) == 1:
        return 'liquid'
    elif phase_name == 'GAS' and len(constituents) == 1:
        return 'gas'
    elif len(constituents) == 1 or \
        (len(constituents) == 2 and len(constituents[1]) == 1 and list(constituents[1])[0] == Species('VA')):
        return 'solid'
    elif all(len(x) == 1 for x in constituents):
        return 'intermetallic'
    else:
        return 'complex'

def extract_coefficients(expr):
    polys = [v.T*sympy.log(v.T), sympy.log(v.T)] + [v.T**n for n in range(-30,30)]
    result = {}
    for poly in polys:
        coef = expr.coeff(poly)
        if coef is not sympy.S.Zero:
            result[poly] = coef
        expr = expr - poly*coef
    expr = expr.n()
    if isinstance(expr, (sympy.Float, sympy.Integer)):
        result[1] = expr
        expr = sympy.S.Zero
    if expr is not sympy.S.Zero:
        raise ValueError(expr)
    return result

total_results = []

for combo in itertools.combinations(dbf.elements - {'VA', '/-'}, 2):
    combo_str = sorted(set(combo) | {'VA'})
    generic_combo = list(string.ascii_uppercase[:len(combo)])
    combo = [Species(x) for x in combo_str]
    combo_dict = dict(zip([c for c in combo if c.number_of_atoms > 0], string.ascii_uppercase))
    combo_dict[Species('VA')] = 'VA'
    allowed_phases = set()
    # TODO: Categorize each phase? solid solution, intermetallic, complex intermetallic, liquid, etc.
    for phase_name, phase_obj in dbf.phases.items():
        include_phase = True
        for sublattice in phase_obj.constituents:
            if len(set(sublattice).intersection(combo)) == 0:
                # None of the components in a sublattice are active
                # We cannot build a model of this phase
                include_phase = False
                break
        if include_phase:
            allowed_phases |= {phase_name}
    for param in dbf._parameters.all():
        if param['phase_name'] in allowed_phases and param['parameter_type'] in ['G', 'L']:
            active_pure_elements = [list(x.constituents.keys()) for subl in param['constituent_array'] for x in subl]
            active_pure_elements = [el.upper() for constituents in active_pure_elements for el in constituents]
            active_pure_elements = set(active_pure_elements)
            if not active_pure_elements.issubset(combo_str):
                continue
            ratio_sum = sum(dbf.phases[param['phase_name']].sublattices)
            reduced_ratios = tuple(sigfigs(c, 3) for c in dbf.phases[param['phase_name']].sublattices)
            param_val = param['parameter'].subs(dbf.symbols).subs(dbf.symbols)
            # If parameter is Piecewise, average it
            pw_atoms = param_val.atoms(sympy.Piecewise)
            pw_dict = {key: average_piecewise(key) for key in pw_atoms}
            param_val = param_val.subs(pw_dict)
            columns = ['system', 'phase_name', 'phase_type', 'site_ratios',
                       'parameter_order', 'constituent_array', 'parameter_value', 'extracted_coefficients']
            id_tuple = (tuple(c for c in combo_str if c != 'VA'), param['phase_name'],
                        categorize_phase(param['phase_name'], dbf.phases[param['phase_name']].constituents, reduced_ratios),
                        reduced_ratios, param['parameter_order'],
                        genericize(combo_dict, param['constituent_array']),
                        param_val,
                        extract_coefficients(param_val)
                       )
            total_results.append(id_tuple)

In [3]:
import pandas as pd
from IPython.display import display

df = pd.DataFrame(total_results, columns=columns)
display(df)

,system,phase_name,phase_type,site_ratios,parameter_order,constituent_array,parameter_value,extracted_coefficients
0,"(AL, CU)",LIQUID,liquid,"(1.0,)",0,"((A,),)",3.96685e-20*T**7 - 8.30236375e-7*T**3 + 0.0020...,"{T**3: -8.30236375000000e-7, T**7: 3.966850000..."
1,"(AL, CU)",LIQUID,liquid,"(1.0,)",0,"((B,),)",-1.94963333333333e-21*T**7 + 1.43581111111111e...,"{T**3: 1.43581111111111e-8, T**7: -1.949633333..."
2,"(AL, CU)",LIQUID,liquid,"(1.0,)",0,"((A, B),)",4.2775*T - 33547,"{1: -33547.0000000000, T: 4.27750000000000}"
3,"(AL, CU)",LIQUID,liquid,"(1.0,)",1,"((A, B),)",-3.559*T + 16074,"{1: 16074.0000000000, T: -3.55900000000000}"
4,"(AL, CU)",LIQUID,liquid,"(1.0,)",2,"((A, B),)",-2.9445*T + 5915/2,"{1: 2957.50000000000, T: -2.94450000000000}"
5,"(AL, CU)",LIQUID,liquid,"(1.0,)",3,"((A, B),)",2.75*T - 3645,"{1: -3645.00000000000, T: 2.75000000000000}"
6,"(AL, CU)",AL2CU,complex,"(2.0, 1.0)",0,"((A,), (A,))",-1.2453545625e-6*T**3 + 0.0031213725*T**2 - 17...,"{T**3: -1.24535456250000e-6, T: 99.29505693750..."
7,"(AL, CU)",AL2CU,complex,"(2.0, 1.0)",0,"((A,), (B,))",-1.63893558333333e-6*T**3 + 0.0037190233333333...,"{T**3: -1.63893558333333e-6, T: 193.5879197500..."
8,"(AL, CU)",AL2CU,complex,"(2.0, 1.0)",0,"((A,), (A, B))",2211/2,{1: 1105.50000000000}
9,"(AL, CU)",AL5CU8,intermetallic,"(5.0, 8.0)",0,"((A,), (B,))",-3.97888454166667e-6*T**3 + 0.0068621216666666...,"{T**3: -3.97888454166667e-6, T: 764.6681071250..."


In [4]:
for system_name, system_df in df.groupby('system'):
    system_df = system_df.drop_duplicates(subset='phase_name')
    print('-')
    print('-'.join(system_name))
    print(system_df['phase_type'].value_counts().to_string())

-
AL-CU
complex          7
solid            5
intermetallic    3
liquid           1
-
AL-ZR
complex    20
solid       6
liquid      1
-
CU-ZR
complex    25
solid       4
liquid      1
